# step 1: install required libraries
# before we begin, install the necessary libraries if they are not already installed.


In [38]:
#!pip install -q transformers torch
#!pip install transformers torch textblob


# step 2: import required modules

In [49]:

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from textblob import TextBlob


# step 3: load pretrained model and tokenizer


In [40]:
# we use a pretrained gpt-2 model for our rlhf experiment.
tokenizer = AutoTokenizer.from_pretrained("gpt2")
policy_model = AutoModelForCausalLM.from_pretrained("gpt2")


In [41]:
# define pad token explicitly to avoid warnings

def generate_completion(prompt, max_length=50):
    input_ids = tokenizer.encode(prompt, return_tensors="pt")
    output = policy_model.generate(input_ids, max_length=max_length, do_sample=True, top_k=50, top_p=0.95)
    return tokenizer.decode(output[0], skip_special_tokens=True)

# Example usage
sample_prompt = "Once upon a time,"
sample_completion = generate_completion(sample_prompt)
print("Generated Completion:", sample_completion)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated Completion: Once upon a time, I've lost my faith in the power of God, and if I do not believe my faith and my work are well balanced, who will know what purpose this office is for me, or whether I will ever be able to


# step 4: generate sample completions


In [55]:
# define pad token explicitly to avoid warnings
tokenizer.pad_token = tokenizer.eos_token
policy_model.config.pad_token_id = tokenizer.pad_token_id

# let's define a function to generate text completions from our model.
# def generate_completion(prompt, max_length=50):
#     input_ids = tokenizer.encode(prompt, return_tensors="pt")
#     attention_mask = input_ids.ne(tokenizer.pad_token_id).long()
#     output = policy_model.generate(
#         input_ids,
#         attention_mask=attention_mask,
#         max_length=max_length,
#         do_sample=True,
#         top_k=50,
#         top_p=0.95
#     )
#     return tokenizer.decode(output[0], skip_special_tokens=True)

# # example usage
# sample_prompt = "once upon a time,"
# sample_completion = generate_completion(sample_prompt)
# #print("generated completion:", sample_completion)
# import textwrap

# print("Generated Completion:")
# print(textwrap.fill(sample_completion.strip(), width=60))  # Simulate whiteboard width

def generate_multiple_completions(prompt, num_samples=3, max_length=50):
    completions = []
    for _ in range(num_samples):
        input_ids = tokenizer.encode(prompt, return_tensors="pt")
        attention_mask = input_ids.ne(tokenizer.pad_token_id).long()
        output = policy_model.generate(
            input_ids,
            attention_mask=attention_mask,
            max_length=max_length,
            do_sample=True,
            top_k=50,
            top_p=0.95
        )
        completions.append(tokenizer.decode(output[0], skip_special_tokens=True))
    return completions


# step 5: implementing a simple reward function


In [50]:
# # now, let's define a simple reward function that scores generated text based on certain criteria.
# def reward_function(text):
#     """
#     a simple reward function that assigns a score based on politeness.
#     returns +1 if the text contains polite phrases, -1 otherwise.
#     """
#     polite_phrases = ["thank you", "please", "kindly", "appreciate"]
#     if any(phrase in text.lower() for phrase in polite_phrases):
#         return 1.0
#     return -1.0

# # example usage
# reward_score = reward_function(sample_completion)
# print("reward score:", reward_score)


def reward_function(text):
    """
    an improved reward function that assigns a score based on politeness and sentiment.
    """
    polite_phrases = ["thank you", "please", "kindly", "appreciate", "grateful", "respect"]
    sentiment = TextBlob(text).sentiment.polarity

    if any(phrase in text.lower() for phrase in polite_phrases):
        return 2.0  # increased reward for polite phrases
    elif sentiment > 0.2:
        return 1.0  # slightly positive sentiment gets a moderate reward
    return -2.0  # increased penalty for non-positive responses

# step 6: computing log probabilities for the generated text


In [44]:
# now, let's define a function to compute log probabilities of the generated tokens.
def compute_log_probs(model, input_text):
    """
    computes log probabilities of the generated text under the given model.
    """
    input_ids = tokenizer.encode(input_text, return_tensors="pt")
    with torch.no_grad():
        outputs = model(input_ids, labels=input_ids)
    log_probs = -outputs.loss.item()
    return log_probs

# example usage
log_prob_score = compute_log_probs(policy_model, sample_completion)
print("log probability score:", log_prob_score)


log probability score: -2.65922474861145


# step 7: computing the advantage estimate


In [45]:
# the advantage function helps determine how much better (or worse) an action was compared to the expected value.
def compute_advantage(reward, baseline=0.0):
    """
    computes the advantage estimate given the reward and an optional baseline.
    """
    return reward - baseline

# # example usage
# advantage = compute_advantage(reward_score)
# print("advantage estimate:", advantage)


advantage estimate: -1.0


# step 8: computing the policy gradient loss


In [51]:
# now, let's compute the policy gradient loss using the advantage estimate.
# now, let's compute the policy gradient loss using the advantage estimate.
def compute_policy_gradient_loss(log_prob, advantage):
    """
    computes the policy gradient loss given log probabilities and advantage.
    """
    return torch.tensor(-log_prob * advantage, requires_grad=True)


# example usage
policy_gradient_loss = compute_policy_gradient_loss(log_prob_score, advantage)
print("policy gradient loss:", policy_gradient_loss)


policy gradient loss: tensor(-2.6592, requires_grad=True)


# step 9: updating the model using gradient descent


In [52]:
# now, let's update the model parameters using the computed loss.
def update_model(model, loss, learning_rate=1e-5):
    """
    updates the model parameters using gradient descent.
    """
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

# example usage
update_model(policy_model, policy_gradient_loss)
print("model updated successfully.")


model updated successfully.


# step 10: evaluating the updated model


In [56]:
# # now, let's generate a new sample completion after training to see if the model has improved.
# def evaluate_model(prompt):
#     """
#     generates a new completion after model updates to evaluate changes.
#     """
#     new_completion = generate_completion(prompt)
#     new_reward = reward_function(new_completion)
#     print("new generated completion:", new_completion)
#     print("new reward score:", new_reward)
#     return new_completion, new_reward

# # example usage
# evaluate_model("once upon a time,")

# def train_model(model, prompt, iterations=30):
#     for i in range(iterations):
#         print(f"iteration {i+1}:")
#         completion = generate_completion(prompt)
#         reward = reward_function(completion)
#         log_prob = compute_log_probs(model, completion)
#         advantage = compute_advantage(reward)
#         loss = compute_policy_gradient_loss(log_prob, advantage)
#         update_model(model, loss)
#         print("updated model.")

# # example usage
# train_model(policy_model, "once upon a time,")


def train_model(model, prompts, iterations=30, num_samples=3):
    """
    trains the model on multiple prompts and multiple generated responses per prompt.
    """
    for i in range(iterations):
        print(f"iteration {i+1}:")
        total_loss = 0
        for prompt in prompts:
            completions = generate_multiple_completions(prompt, num_samples)
            rewards = [reward_function(c) for c in completions]
            avg_reward = sum(rewards) / len(rewards)
            log_probs = [compute_log_probs(model, c) for c in completions]
            avg_log_prob = sum(log_probs) / len(log_probs)
            advantage = compute_advantage(avg_reward)
            loss = compute_policy_gradient_loss(avg_log_prob, advantage)
            update_model(model, loss)
            total_loss += loss.item()
        print(f"updated model. average loss: {total_loss / len(prompts)}")

# example usage
training_prompts = [
    "once upon a time,",
    "hello, how are you doing today?",
    "thank you for your help!"
]
train_model(policy_model, training_prompts, iterations=30)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


iteration 1:


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


updated model. average loss: -0.47951141993204754
iteration 2:


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


...

updated model. average loss: 0.7708389759063721
iteration 29:


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


updated model. average loss: 1.60145370165507
iteration 30:


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


updated model. average loss: -1.0149595737457275


# step 11: evaluating the updated model


In [57]:
def evaluate_model(prompt):
    """
    generates a new completion after model updates to evaluate changes.
    """
    new_completion = generate_completion(prompt)
    new_reward = reward_function(new_completion)
    print("new generated completion:", new_completion)
    print("new reward score:", new_reward)
    return new_completion, new_reward

# example usage
evaluate_model("once upon a time,")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


new generated completion: once upon a time, the world could not possibly have suffered as catastrophically a situation as the one in Egypt, which at the time the British considered to be the worst-kept secret. The United States had been doing its utmost to convince Germany
new reward score: -2.0


('once upon a time, the world could not possibly have suffered as catastrophically a situation as the one in Egypt, which at the time the British considered to be the worst-kept secret. The United States had been doing its utmost to convince Germany',
 -2.0)

# Putting all together

In [2]:
# step 1: install required libraries
# before we begin, install the necessary libraries if they are not already installed.
#!pip install transformers torch textblob

# step 2: import required modules
import torch
import random
from transformers import AutoTokenizer, AutoModelForCausalLM
from textblob import TextBlob

# step 3: load pretrained model and tokenizer
# we use a pretrained gpt-2 model for our rlhf experiment.
tokenizer = AutoTokenizer.from_pretrained("gpt2")
policy_model = AutoModelForCausalLM.from_pretrained("gpt2")

# define pad token explicitly to avoid warnings
tokenizer.pad_token = tokenizer.eos_token
policy_model.config.pad_token_id = tokenizer.pad_token_id

# step 4: generate multiple sample completions
def generate_multiple_completions(prompt, num_samples=3, max_length=50):
    completions = []
    for _ in range(num_samples):
        input_ids = tokenizer.encode(prompt, return_tensors="pt")
        attention_mask = input_ids.ne(tokenizer.pad_token_id).long()
        output = policy_model.generate(
            input_ids,
            attention_mask=attention_mask,
            max_length=max_length,
            do_sample=True,
            top_k=50,
            top_p=0.95,
            temperature=1.2  # encourage exploration
        )
        completions.append(tokenizer.decode(output[0], skip_special_tokens=True))
    return completions

# step 5: refining the reward function with noise injection
def reward_function(text):
    """
    a refined reward function that assigns a progressive score based on politeness and sentiment intensity,
    with random noise to encourage policy updates.
    """
    polite_phrases = ["thank you", "please", "kindly", "appreciate", "grateful", "respect", "much obliged", "thanks"]
    sentiment = TextBlob(text).sentiment.polarity

    # progressive scoring
    if any(phrase in text.lower() for phrase in polite_phrases):
        reward = 2.0  # strong positive reward for explicit politeness
    elif sentiment > 0.5:
        reward = 1.5  # strong positive sentiment
    elif sentiment > 0.2:
        reward = 1.0  # moderate positive sentiment
    elif sentiment > 0.0:
        reward = 0.5  # slight positive sentiment
    elif sentiment < -0.5:
        reward = -2.0  # strong penalty for highly negative sentiment
    else:
        reward = -1.0  # default penalty for neutral or slightly negative sentiment

    # add small noise to ensure variation in rewards
    return reward + random.uniform(-0.2, 0.2)

# step 6: computing log probabilities for the generated text
def compute_log_probs(model, input_text):
    input_ids = tokenizer.encode(input_text, return_tensors="pt")
    with torch.no_grad():
        outputs = model(input_ids, labels=input_ids)
    log_probs = -outputs.loss.item()
    return log_probs

# step 7: improving advantage computation to prevent zero values
def compute_advantage(reward, reward_baseline=0.0, epsilon=1e-6):
    advantage = (reward - reward_baseline) / (abs(reward_baseline) + epsilon)
    return max(advantage, 1e-3)  # prevent zero advantage values

# step 8: computing the policy gradient loss
def compute_policy_gradient_loss(log_prob, advantage):
    return torch.tensor(-log_prob * advantage, requires_grad=True)

# step 9: updating the model using gradient descent
def update_model(model, loss, learning_rate=1e-5):
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

# step 10: training loop with debugging outputs
def train_model(model, prompts, iterations=30, num_samples=3):
    """
    trains the model on multiple prompts and multiple generated responses per prompt using progressive rewards and normalized advantage.
    """
    for i in range(iterations):
        print(f"iteration {i+1}:")
        total_loss = 0
        reward_baseline = 0  # baseline initialized per iteration
        for prompt in prompts:
            completions = generate_multiple_completions(prompt, num_samples)
            rewards = [reward_function(c) for c in completions]
            reward_baseline = sum(rewards) / len(rewards)  # update baseline dynamically
            log_probs = [compute_log_probs(model, c) for c in completions]
            avg_reward = sum(rewards) / len(rewards)
            avg_log_prob = sum(log_probs) / len(log_probs)
            advantage = compute_advantage(avg_reward, reward_baseline)
            loss = compute_policy_gradient_loss(avg_log_prob, advantage)
            update_model(model, loss)
            total_loss += loss.item()

            # debugging outputs
            print(f"prompt: {prompt}")
            print(f"generated completions: {completions}")
            print(f"rewards: {rewards}, baseline: {reward_baseline}")
            print(f"log probabilities: {log_probs}")
            print(f"computed advantage: {advantage}")
            print(f"computed loss: {loss.item()}")
            print("-" * 50)

        print(f"updated model. average loss: {total_loss / len(prompts)}")

# example usage
training_prompts = [
    "once upon a time,",
    "hello, how are you doing today?",
    "thank you for your help!",
    "i really appreciate your kindness.",
    "could you please assist me with this?"
]
train_model(policy_model, training_prompts, iterations=30)




   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 78.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 81.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 86.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


iteration 1:


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


prompt: once upon a time,
generated completions: ['once upon a time, but at least one of this is happening."\n\nThe study was published today (April 5th) in the journal Cell and will be followed by a later release that includes more on the link to these researchers and some of', 'once upon a time, you will be able to see him as one who always does his homework and always remembers what he likes."\n\nAnd that\'s in what he means on a personal level, too: "I\'m very much a father figure', 'once upon a time, when there are only four people for many who are in danger for each other and yet all are capable of caring for one another. But there are still more to a relationship that does not depend upon some sort of force or power']
rewards: [0.563179194320097, 0.8661526295122776, 1.0495142163934765], baseline: 0.826282013408617
log probabilities: [-2.9378597736358643, -2.8032310009002686, -3.1920664310455322]
computed advantage: 0.001
computed loss: 0.0029777190648019314
---------------------

...

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


prompt: i really appreciate your kindness.
generated completions: ["i really appreciate your kindness.\n\nAs someone that gets most of his credit due to his dedication to the cause of the planet, I have been involved in many missions to support the cause on various levels of the Earth. I've never had anyone", "i really appreciate your kindness.\n\nBut there's really never a day you can't make dinner for your loved one that day anyway - it doesn't matter. We'll just leave it at that for now.\n\nThe second best thing:", "i really appreciate your kindness. Do you have any special wishes for her?\n\n[Mimicked by some of her fans on Twitter.] [Mimicked by some of her fans on Twitter.] You sure don't mind our little joke,"]
rewards: [1.9620913401244005, 2.1898843641796546, 2.1777629515707706], baseline: 2.1099128852916085
log probabilities: [-3.0134124755859375, -2.9588654041290283, -2.598257064819336]
computed advantage: 0.001
computed loss: 0.002856845036149025
----------------------------

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


prompt: could you please assist me with this?
generated completions: ['could you please assist me with this?[/b] I\'ll give you my time in creating it. I will make use of all of your generosity for the betterment of all."\n\nAfter his final performance with the Red Sox, Johnson said', 'could you please assist me with this?', 'could you please assist me with this? Thank you." "Why don\'t you go down to my room?" *shrug* "Yeah, um, it\'ll be cool to go, I guess. Do you have a room nearby." >I']
rewards: [2.1296931902060314, 1.836295977507978, 1.8192256504948843], baseline: 1.9284049394029645
log probabilities: [-2.950019598007202, -3.1072323322296143, -2.8005764484405518]
computed advantage: 0.001
computed loss: 0.00295260944403708
--------------------------------------------------
updated model. average loss: 0.002840976510196924
new generated completion: once upon a time, he might take it upon himself to put the burden on his brother's shoulders. That was a lesson that the king had fai

("once upon a time, he might take it upon himself to put the burden on his brother's shoulders. That was a lesson that the king had failed to take into account, as well as a lesson that he was capable of taking up the matter for",
 -1.0995761423599568)

In [3]:
# step 11: evaluating the updated model
def evaluate_model(prompt):
    new_completion = generate_multiple_completions(prompt, num_samples=1)[0]
    new_reward = reward_function(new_completion)
    print("new generated completion:", new_completion)
    print("new reward score:", new_reward)
    return new_completion, new_reward

# example usage
evaluate_model("once upon a time,")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


new generated completion: once upon a time, I didn't know that I existed." When his father asked if he was still alive he replied, "Maybe someday. But now, I can't look at it anymore." Even though he has no interest in his life,
new reward score: 0.48848573721266714


('once upon a time, I didn\'t know that I existed." When his father asked if he was still alive he replied, "Maybe someday. But now, I can\'t look at it anymore." Even though he has no interest in his life,',
 0.48848573721266714)

# Refining the reward system further
I will:

introduce a negative word penalty
– penalize words like "no", "can't", "won't", "not", "bad", which contribute to negative sentiment.
increase reward gap for polite vs. non-polite responses
– to encourage stronger preference learning.
strengthen the reinforcement learning signal
 – by adjusting the advantage calculation.
i'll update the notebook now. then, run the updated version and check the evaluation output again.



# # Introduce a penalty:

 for negative words like "no", "can't", "not", which contribute to negative sentiment.
increase the reward gap to further encourage positive completions.
amplify the reinforcement signal by tweaking the advantage function.

In [4]:
# step 1: install required libraries
# before we begin, install the necessary libraries if they are not already installed.
#!pip install transformers torch textblob

# step 2: import required modules
import torch
import random
from transformers import AutoTokenizer, AutoModelForCausalLM
from textblob import TextBlob

# step 3: load pretrained model and tokenizer
# we use a pretrained gpt-2 model for our rlhf experiment.
tokenizer = AutoTokenizer.from_pretrained("gpt2")
policy_model = AutoModelForCausalLM.from_pretrained("gpt2")

# define pad token explicitly to avoid warnings
tokenizer.pad_token = tokenizer.eos_token
policy_model.config.pad_token_id = tokenizer.pad_token_id

# step 4: generate multiple sample completions
def generate_multiple_completions(prompt, num_samples=3, max_length=50):
    completions = []
    for _ in range(num_samples):
        input_ids = tokenizer.encode(prompt, return_tensors="pt")
        attention_mask = input_ids.ne(tokenizer.pad_token_id).long()
        output = policy_model.generate(
            input_ids,
            attention_mask=attention_mask,
            max_length=max_length,
            do_sample=True,
            top_k=50,
            top_p=0.95,
            temperature=1.2  # encourage exploration
        )
        completions.append(tokenizer.decode(output[0], skip_special_tokens=True))
    return completions

# step 5: refining the reward function with stronger penalization
def reward_function(text):
    """
    a refined reward function that assigns a progressive score based on politeness and sentiment intensity,
    with a stronger penalty for negative words.
    """
    polite_phrases = ["thank you", "please", "kindly", "appreciate", "grateful", "respect", "much obliged", "thanks"]
    negative_words = ["no", "not", "never", "can't", "won't", "bad", "sad", "problem", "worse"]
    sentiment = TextBlob(text).sentiment.polarity

    # progressive scoring
    if any(phrase in text.lower() for phrase in polite_phrases):
        reward = 3.0  # increased reward for strong politeness
    elif sentiment > 0.5:
        reward = 2.0  # strong positive sentiment
    elif sentiment > 0.2:
        reward = 1.5  # moderate positive sentiment
    elif sentiment > 0.0:
        reward = 1.0  # slight positive sentiment
    elif sentiment < -0.5:
        reward = -3.0  # increased penalty for highly negative sentiment
    else:
        reward = -1.5  # default penalty for neutral or slightly negative sentiment

    # additional penalty for explicit negative words
    if any(word in text.lower() for word in negative_words):
        reward -= 1.0

    # add small noise to ensure variation in rewards
    return reward + random.uniform(-0.2, 0.2)

# step 6: computing log probabilities for the generated text
def compute_log_probs(model, input_text):
    input_ids = tokenizer.encode(input_text, return_tensors="pt")
    with torch.no_grad():
        outputs = model(input_ids, labels=input_ids)
    log_probs = -outputs.loss.item()
    return log_probs

# step 7: improving advantage computation to amplify differences
def compute_advantage(reward, reward_baseline=0.0, epsilon=1e-6):
    advantage = (reward - reward_baseline) / (abs(reward_baseline) + epsilon)
    return max(advantage, 1e-2)  # prevent zero advantage values

# step 8: computing the policy gradient loss
def compute_policy_gradient_loss(log_prob, advantage):
    return torch.tensor(-log_prob * advantage, requires_grad=True)

# step 9: updating the model using gradient descent
def update_model(model, loss, learning_rate=1e-5):
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

# step 10: training loop with debugging outputs
def train_model(model, prompts, iterations=30, num_samples=3):
    """
    trains the model on multiple prompts and multiple generated responses per prompt using progressive rewards and normalized advantage.
    """
    for i in range(iterations):
        print(f"iteration {i+1}:")
        total_loss = 0
        reward_baseline = 0  # baseline initialized per iteration
        for prompt in prompts:
            completions = generate_multiple_completions(prompt, num_samples)
            rewards = [reward_function(c) for c in completions]
            reward_baseline = sum(rewards) / len(rewards)  # update baseline dynamically
            log_probs = [compute_log_probs(model, c) for c in completions]
            avg_reward = sum(rewards) / len(rewards)
            avg_log_prob = sum(log_probs) / len(log_probs)
            advantage = compute_advantage(avg_reward, reward_baseline)
            loss = compute_policy_gradient_loss(avg_log_prob, advantage)
            update_model(model, loss)
            total_loss += loss.item()

            # debugging outputs
            print(f"prompt: {prompt}")
            print(f"generated completions: {completions}")
            print(f"rewards: {rewards}, baseline: {reward_baseline}")
            print(f"log probabilities: {log_probs}")
            print(f"computed advantage: {advantage}")
            print(f"computed loss: {loss.item()}")
            print("-" * 50)

        print(f"updated model. average loss: {total_loss / len(prompts)}")

# example usage
training_prompts = [
    "once upon a time,",
    "hello, how are you doing today?",
    "thank you for your help!",
    "i really appreciate your kindness.",
    "could you please assist me with this?"
]
train_model(policy_model, training_prompts, iterations=30)




Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


iteration 1:


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


prompt: once upon a time,
generated completions: ['once upon a time, it was an opportunity to have some fun, a good time and a decent food, to let everyone know they were safe and secure."\n\nOn how she got to where she is today and how that will happen: "', 'once upon a time, when human civilization seemed dead to their minds. And where were all those brave souls, for all the terrible crimes and crimes of civilization, before they all perished and found themselves among us?\n\nAnd where are these dark souls', "once upon a time, we can learn about our own. And I think some will ask:\n\nWhat about what is really good and how do we know we're good?\n\nWhat does that have to do with who we are and how"]
rewards: [0.4491432386347946, -1.3670078733520508, 0.8752388014605408], baseline: -0.014208611085571773
log probabilities: [-2.7610368728637695, -3.281049966812134, -2.587794780731201]
computed advantage: 0.01
computed loss: 0.028766272589564323
--------------------------------------------

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


...

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


prompt: i really appreciate your kindness.
generated completions: ["i really appreciate your kindness. I'm sure that your gift will be appreciated by others too and that we can have a lovely couple together together.\n\nAnd my dear, so I just thought I had all of you down again. So there you", "i really appreciate your kindness. For instance: a beautiful lady in a flowerbed who didn't dare bring her children under her wing, and she came to tell me not to see them. Her family said not to have them, and they didn't", 'i really appreciate your kindness.\n\nThis post first appeared at www.truenews.com\n\nRead or Share this story: https://cin.ci/2i7FzcXa']
rewards: [3.131484179374063, 2.176358498987463, 2.8970432237524735], baseline: 2.7349619673713335
log probabilities: [-3.168627977371216, -3.3931539058685303, -2.4720804691314697]
computed advantage: 0.01
computed loss: 0.030112873762845993
--------------------------------------------------


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


prompt: could you please assist me with this?
generated completions: ['could you please assist me with this?\n\nIt appears to be a problem between two different phones (not the HTC One and the one we currently have). And, from my test case that one of the phones is an LG phone, I have', 'could you please assist me with this? As of yet I am in need of money to buy a new pair of pants. I should know about your expenses soon, my name is Zuze. I had a call for you while I was sleeping', 'could you please assist me with this? I will probably spend more money in different parts of Europe," said Mavrodi.\n\nThe Dutch and British foreign service were unable to reach one another. Mavrodi told the Guardian that his']
rewards: [1.8465395376877463, 1.8739611135333927, 2.0525245951620414], baseline: 1.9243417487943935
log probabilities: [-3.1084439754486084, -2.9692630767822266, -2.9362149238586426]
computed advantage: 0.01
computed loss: 0.030046407133340836
---------------------------------------

('once upon a time, that the first two generations to arouse the greatest spirit of love have fallen in the manner of a fall; and I pray and pray that this great spirit which I share, and I testify this to the whole of you by',
 1.97042514646537)

In [5]:
# step 11: evaluating the updated model
def evaluate_model(prompt):
    new_completion = generate_multiple_completions(prompt, num_samples=1)[0]
    new_reward = reward_function(new_completion)
    print("new generated completion:", new_completion)
    print("new reward score:", new_reward)
    return new_completion, new_reward

# example usage
evaluate_model("once upon a time,")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


new generated completion: once upon a time, we do not know whether the human brain is the source or the repository of the entire experience or whether, after the events of certain historical periods, it is always the same: the individual feeling, the instinct; the process of
new reward score: 0.135034022921793


('once upon a time, we do not know whether the human brain is the source or the repository of the entire experience or whether, after the events of certain historical periods, it is always the same: the individual feeling, the instinct; the process of',
 0.135034022921793)

# DeepSeek moodel

In [23]:
from huggingface_hub import login
login()


In [ ]:
# step 1: install required libraries
#!pip install transformers torch textblob bitsandbytes

# step 2: import required modules
import torch
import random
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from textblob import TextBlob

# step 3: configure quantization and load model
# Create a simple 8-bit quantization config
quantization_config = {
    "quant_method": "bitsandbytes_8bit"
}

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("deepseek-ai/DeepSeek-R1", trust_remote_code=True)
policy_model = AutoModelForCausalLM.from_pretrained(
    "deepseek-ai/DeepSeek-R1",
    trust_remote_code=True,
    device_map="auto",
    load_in_8bit=True,  # Enable 8-bit quantization directly
    quantization_config=quantization_config
)

# define pad token explicitly to avoid warnings
tokenizer.pad_token = tokenizer.eos_token
policy_model.config.pad_token_id = tokenizer.pad_token_id

# Rest of your RLHF implementation remains the same...
def generate_multiple_completions(prompt, num_samples=3, max_length=50):
    completions = []
    for _ in range(num_samples):
        input_ids = tokenizer.encode(prompt, return_tensors="pt")
        attention_mask = input_ids.ne(tokenizer.pad_token_id).long()
        output = policy_model.generate(
            input_ids,
            attention_mask=attention_mask,
            max_length=max_length,
            do_sample=True,
            top_k=40,
            top_p=0.92,
            temperature=1.5
        )
        completions.append(tokenizer.decode(output[0], skip_special_tokens=True))
    return completions


# step 5: refining the reward function with stronger penalization
def reward_function(text):
    """
    a refined reward function that assigns a progressive score based on politeness and sentiment intensity,
    with a stronger penalty for negative words.
    """
    polite_phrases = ["thank you", "please", "kindly", "appreciate", "grateful", "respect", "much obliged", "thanks"]
    negative_words = ["no", "not", "never", "can't", "won't", "bad", "sad", "problem", "worse"]
    sentiment = TextBlob(text).sentiment.polarity

    # progressive scoring
    if any(phrase in text.lower() for phrase in polite_phrases):
        reward = 3.0  # increased reward for strong politeness
    elif sentiment > 0.5:
        reward = 2.0  # strong positive sentiment
    elif sentiment > 0.2:
        reward = 1.5  # moderate positive sentiment
    elif sentiment > 0.0:
        reward = 1.0  # slight positive sentiment
    elif sentiment < -0.5:
        reward = -3.0  # increased penalty for highly negative sentiment
    else:
        reward = -1.5  # default penalty for neutral or slightly negative sentiment

    # additional penalty for explicit negative words
    if any(word in text.lower() for word in negative_words):
        reward -= 1.0

    # add small noise to ensure variation in rewards
    return reward + random.uniform(-0.2, 0.2)

# step 6: computing log probabilities for the generated text
def compute_log_probs(model, input_text):
    input_ids = tokenizer.encode(input_text, return_tensors="pt")
    with torch.no_grad():
        outputs = model(input_ids, labels=input_ids)
    log_probs = -outputs.loss.item()
    return log_probs

# step 7: improving advantage computation to amplify differences
def compute_advantage(reward, reward_baseline=0.0, epsilon=1e-6):
    advantage = (reward - reward_baseline) / (abs(reward_baseline) + epsilon)
    return max(advantage, 1e-2)  # prevent zero advantage values

# step 8: computing the policy gradient loss
def compute_policy_gradient_loss(log_prob, advantage):
    return torch.tensor(-log_prob * advantage, requires_grad=True)

# step 9: updating the model using gradient descent
def update_model(model, loss, learning_rate=1e-5):
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

# step 10: training loop with debugging outputs
def train_model(model, prompts, iterations=5, num_samples=3):
    """
    trains the model on multiple prompts and multiple generated responses per prompt using progressive rewards and normalized advantage.
    """
    for i in range(iterations):
        print(f"iteration {i+1}:")
        total_loss = 0
        reward_baseline = 0  # baseline initialized per iteration
        for prompt in prompts:
            completions = generate_multiple_completions(prompt, num_samples)
            rewards = [reward_function(c) for c in completions]
            reward_baseline = sum(rewards) / len(rewards)  # update baseline dynamically
            log_probs = [compute_log_probs(model, c) for c in completions]
            avg_reward = sum(rewards) / len(rewards)
            avg_log_prob = sum(log_probs) / len(log_probs)
            advantage = compute_advantage(avg_reward, reward_baseline)
            loss = compute_policy_gradient_loss(avg_log_prob, advantage)
            update_model(model, loss)
            total_loss += loss.item()

            # debugging outputs
            print(f"prompt: {prompt}")
            print(f"generated completions: {completions}")
            print(f"rewards: {rewards}, baseline: {reward_baseline}")
            print(f"log probabilities: {log_probs}")
            print(f"computed advantage: {advantage}")
            print(f"computed loss: {loss.item()}")
            print("-" * 50)

        print(f"updated model. average loss: {total_loss / len(prompts)}")

# example usage
training_prompts = [
    "once upon a time,",
    "hello, how are you doing today?",
    "thank you for your help!",
    "i really appreciate your kindness.",
    "could you please assist me with this?"
]
train_model(policy_model, training_prompts, iterations=30)




In [ ]:
# step 11: evaluating the updated model
def evaluate_model(prompt):
    new_completion = generate_multiple_completions(prompt, num_samples=1)[0]
    new_reward = reward_function(new_completion)
    print("new generated completion:", new_completion)
    print("new reward score:", new_reward)
    return new_completion, new_reward

# example usage
evaluate_model("once upon a time,")

In [36]:
# step 1: install required libraries
# before we begin, install the necessary libraries if they are not already installed.
#!pip install transformers torch textblob

# step 2: import required modules
import torch
import random
from transformers import AutoTokenizer, AutoModelForCausalLM
from textblob import TextBlob

# step 3: load pretrained model and tokenizer
# using gpt2-medium as our base model for rlhf
model_name = "gpt2-medium"
tokenizer = AutoTokenizer.from_pretrained(model_name)
policy_model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto")

# move model to the correct device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
policy_model.to(device)

# define pad token explicitly to avoid warnings
tokenizer.pad_token = tokenizer.eos_token
policy_model.config.pad_token_id = tokenizer.pad_token_id

# step 4: generate multiple sample completions
def generate_multiple_completions(prompt, num_samples=3, max_length=50):
    completions = []
    for _ in range(num_samples):
        input_ids = tokenizer.encode(prompt, return_tensors="pt").to(device)
        attention_mask = input_ids.ne(tokenizer.pad_token_id).long().to(device)
        output = policy_model.generate(
            input_ids.to(device),
            attention_mask=attention_mask.to(device),
            max_length=max_length,
            do_sample=True,
            top_k=40,  # slightly reduced for better control
            top_p=0.92,  # slight reduction to prevent extreme randomness
            temperature=1.5  # further increase exploration
        )
        completions.append(tokenizer.decode(output[0], skip_special_tokens=True))
    return completions

# step 5: refining the reward function with stronger penalization
def reward_function(text):
    """
    a refined reward function that assigns a progressive score based on politeness and sentiment intensity,
    with a stronger penalty for negative words.
    """
    polite_phrases = ["thank you", "please", "kindly", "appreciate", "grateful", "respect", "much obliged", "thanks"]
    negative_words = ["no", "not", "never", "can't", "won't", "bad", "sad", "problem", "worse"]
    sentiment = TextBlob(text).sentiment.polarity

    # progressive scoring
    if any(phrase in text.lower() for phrase in polite_phrases):
        reward = 3.0  # increased reward for strong politeness
    elif sentiment > 0.5:
        reward = 2.0  # strong positive sentiment
    elif sentiment > 0.2:
        reward = 1.5  # moderate positive sentiment
    elif sentiment > 0.0:
        reward = 1.0  # slight positive sentiment
    elif sentiment < -0.5:
        reward = -3.0  # increased penalty for highly negative sentiment
    else:
        reward = -1.5  # default penalty for neutral or slightly negative sentiment

    # additional penalty for explicit negative words
    if any(word in text.lower() for word in negative_words):
        reward -= 1.0

    # add small noise to ensure variation in rewards
    return reward + random.uniform(-0.2, 0.2)

# step 6: computing log probabilities for the generated text
def compute_log_probs(model, input_text):
    input_ids = tokenizer.encode(input_text, return_tensors="pt").to(device)
    with torch.no_grad():
        outputs = model(input_ids.to(device), labels=input_ids.to(device))
    log_probs = -outputs.loss.item()
    return log_probs

# step 7: improving advantage computation to amplify differences
def compute_advantage(reward, reward_baseline=0.0, epsilon=1e-6):
    advantage = (reward - reward_baseline) / (abs(reward_baseline) + epsilon)
    return max(advantage, 1e-2)  # prevent zero advantage values

# step 8: computing the policy gradient loss
def compute_policy_gradient_loss(log_prob, advantage):
    return -log_prob * advantage

# step 9: updating the model using gradient descent
def update_model(model, loss, learning_rate=1e-5):
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

# step 10: training loop with debugging outputs
def train_model(model, prompts, iterations=5, num_samples=3):
    """
    trains the model on multiple prompts and multiple generated responses per prompt
    using progressive rewards and normalized advantage.
    """
    for i in range(iterations):
        print(f"iteration {i+1}:")
        total_loss = 0
        reward_baseline = 0  # baseline initialized per iteration

        for prompt in prompts:
            completions = generate_multiple_completions(prompt, num_samples)
            rewards = [reward_function(c) for c in completions]
            reward_baseline = sum(rewards) / len(rewards)  # update baseline dynamically

            for completion, reward in zip(completions, rewards):
                log_prob = compute_log_probs(model, completion)
                advantage = compute_advantage(reward, reward_baseline)
                loss = compute_policy_gradient_loss(log_prob, advantage)

                if isinstance(loss, torch.Tensor):
                    update_model(model, loss)
                    total_loss += loss.item()

                # debugging outputs
                print(f"prompt: {prompt}")
                print(f"completion: {completion}")
                print(f"reward: {reward:.3f}")
                print(f"log probability: {log_prob:.3f}")
                print(f"advantage: {advantage:.3f}")
                print(f"loss: {loss if isinstance(loss, float) else loss.item():.3f}")
                print("-" * 50)

        avg_loss = total_loss / (len(prompts) * num_samples)
        print(f"average loss for iteration {i+1}: {avg_loss:.3f}")
        print("=" * 80)

# example usage
training_prompts = [
    "once upon a time,",
    "hello, how are you doing today?",
    "thank you for your help!",
    "i really appreciate your kindness.",
    "could you please assist me with this?"
]

# train the model
train_model(policy_model, training_prompts, iterations=3)


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


iteration 1:


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


prompt: once upon a time,
completion: once upon a time, there was once a guy in a big hotel in New York, which is now gone in the suburbs -- or I forget where but it was probably where this place now is. He was wearing something that looked sort of like a
reward: -0.056
log probability: -2.952
advantage: 0.010
loss: 0.030
--------------------------------------------------
prompt: once upon a time,
completion: once upon a time, I was at school as a very popular student, with wonderful ideas and wonderful talents. But, after four of my six months of teaching, they suddenly decided to pull a quick change," Dr. Ebert says, speaking about
reward: 2.016
log probability: -3.416
advantage: 13.310
loss: 45.464
--------------------------------------------------
prompt: once upon a time,
completion: once upon a time, the Church said it was time for women and men everywhere to begin to think critically about who they love—that all the things men were thinking and seeing would ultimately lead to th

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


...

prompt: thank you for your help!
completion: thank you for your help! You are truly appreciated!! :)"
reward: 2.923
log probability: -2.554
advantage: 0.010
loss: 0.026
--------------------------------------------------
prompt: thank you for your help!
completion: thank you for your help! <3<] This file is a single document that contains the details of how to install NVDD. I have split it and made it as easy as possible with screenshots provided in the instructions below.<br><font
reward: 2.985
log probability: -3.092
advantage: 0.010
loss: 0.031
--------------------------------------------------
prompt: thank you for your help!
completion: thank you for your help! It would still require that we continue to invest into expanding our data-driven operations in order to make sure you enjoy using our app!"

For more about Appoquin, click here.
reward: 2.980
log probability: -2.971
advantage: 0.010
loss: 0.030
--------------------------------------------------


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


prompt: i really appreciate your kindness.
completion: i really appreciate your kindness. It's always easy to think about this when it happens, but your feelings, you know the world won't stop until that one woman gets what's hers.

The two are speaking quietly again. They're hugging
reward: 1.975
log probability: -3.482
advantage: 0.010
loss: 0.035
--------------------------------------------------
prompt: i really appreciate your kindness.
completion: i really appreciate your kindness. I appreciate everything that you are doing for my mother today.

Thanks everyone for reading my story! :) I thank my family too."
reward: 3.192
log probability: -3.344
advantage: 0.182
loss: 0.608
--------------------------------------------------
prompt: i really appreciate your kindness.
completion: i really appreciate your kindness. And I promise you something? That will be all. Please keep your mouth shut."

Sasha took her finger and pushed her tongue out of the door of Ruby's apartment. She grabbe

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


prompt: could you please assist me with this?
completion: could you please assist me with this? Are people like us who understand the meaning of this issue truly "racist"? And please try and explain why that is. If you cannot do that, what other services have you implemented to help your readers?

reward: 2.196
log probability: -3.181
advantage: 0.010
loss: 0.032
--------------------------------------------------
prompt: could you please assist me with this?
completion: could you please assist me with this? The information will not be available till after the match".

'If we have got one player, I will let him get there,' said Klopp.

That's when things got real heated.

reward: 2.106
log probability: -3.053
advantage: 0.010
loss: 0.031
--------------------------------------------------
prompt: could you please assist me with this?
completion: could you please assist me with this? It is almost 1-quarter way into an appointment with Doctor Ophira. I will be waiting on my husband. Thanks

In [37]:
# step 11: evaluating the updated model
def evaluate_model(prompt):
    new_completion = generate_multiple_completions(prompt, num_samples=1)[0]
    new_reward = reward_function(new_completion)
    print("new generated completion:", new_completion)
    print("new reward score:", new_reward)
    return new_completion, new_reward

# example usage
evaluate_model("once upon a time,")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


new generated completion: once upon a time, and it was not possible to know that it was going to stop until it was ready. I don't believe, therefore, in the current theory as such." (Emma Goldman - A Woman's Work, Chapter 7)
new reward score: 0.18487955503964265


('once upon a time, and it was not possible to know that it was going to stop until it was ready. I don\'t believe, therefore, in the current theory as such." (Emma Goldman - A Woman\'s Work, Chapter 7)',
 0.18487955503964265)